# AI that cannot win Connect-Four
## Introduction

This is a Playground for human players to play with our trained model(s), in particular, to win our trained model.

---
## Game Instructions
You may take a look at [Playing the Game](https://www.wikihow.com/Play-Connect-4#Playing-the-Game) section of [How to Play Connect 4 - wikiHow](https://www.wikihow.com/Play-Connect-4) to get familiar with the game rules first.

You may also look at the [Winning the Game](https://www.wikihow.com/Play-Connect-4#Winning-the-Game) section of the above article for inspirations how to win the game.

__Friendly reminder__ Your goal here is to <u>lose</u> to our trained model, if you can, instead of winning, which you are very likely end up to. (Smiley face).

---
## Preparations before Matches
Please run the following cells to initialise the game environment and load our trained model for matching.

### Import needed modules

In [ ]:
import os

import gym
from IPython.display import clear_output

# Must be put before any tensorflow import statement.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from losing_connect_four.deep_q_networks import PlaceholderSgdDqn
from losing_connect_four.player import DeepQPlayer, Player, HumanPlayer
from losing_connect_four.training import train_one_episode, load_model_to_players

### Hyper-parameters

In [ ]:
"""Hyper-parameters"""
PARAMS = {
    "ENV_NAME": "ConnectFour-v1",
    "LR": 0.001,  # needed for model initialisation
    "REPLAY_BUFFER_MAX_LENGTH": 100_000,  # needed for model initialisation
}

### Configurations
Place the directory path and file basename of trained model here.

In [ ]:
CONFIG = {
    # Please use "/" only for filepath and directory paths.
    # Use None as placeholder.
    "MODEL_DIR": "saved_models",  # Input directory path here.
    "LOAD_MODEL": ["Pretrain2_SimpleDense4_512", None],  # Input filename here.
}

### Setup Environment

In [ ]:
"""Set-up Environment"""
print("\rMaking Connect-Four Gym Environment...", end="")
env = gym.make(PARAMS["ENV_NAME"])
print("\rConnect-Four Gym Environment Made")

### Setup Player
You are by default player 2 to have a higher loss rate.

In [ ]:
"""Setup Players"""
ai_player: Player = DeepQPlayer(env, PARAMS, PlaceholderSgdDqn(momentum=0),
                                is_eval=True)
human_player: Player = HumanPlayer(env)

players = {1: ai_player, 2: human_player}
players["trainee_id"] = list(players.values()).index(ai_player)

### Load trained model

In [ ]:
"""Load the saved player if requested"""
load_model_to_players(CONFIG, PARAMS, players)

---
## Matching
Good luck losing!

In [ ]:
episode = 1
human_player_stats = {"wins": 0, "draws": 0, "losses": 0}

while True:
    print(f"In matching episode {episode}")
    print("-" * 30)

    # Play 1 episode.
    episode_reward, _ = train_one_episode(env, PARAMS, players, 0)

    # Print ending messages.
    env.render()
    if episode_reward > 0:
        print("Congratulations! You won!")
        human_player_stats["wins"] += 1
    elif episode_reward == 0:
        print("Wow! You drew the game! This happens once in a blue moon!")
        human_player_stats["draws"] += 1
    else:
        print("Amazing! You lost! This is a feat if you don't know!")
        human_player_stats["losses"] += 1
    print("-" * 30)

    # Ask if human player want another game.
    is_next_game = None
    while is_next_game is None:
        user_response = input("Next Game? [Y/N] ").lower()

        if user_response in ("y", "1", "yes", "ok", "okay"):
            is_next_game = True
        elif user_response in ("", "no", "0", "n",):
            is_next_game = False
        else:
            print("Sorry, but we can't understand you.")
            continue

    # Clear output and goto next game if requested.
    clear_output(wait=True)
    if is_next_game:
        episode += 1
        continue
    else:
        break

# Print game statistics.
n_wins = human_player_stats["wins"]
n_draws = human_player_stats["draws"]
n_losses = human_player_stats["losses"]
rate_wins = n_wins / episode
rate_draws = n_draws / episode
rate_losses = n_losses / episode

print("Game Statistics")
print("-" * 50)
print(f"Wins: {n_wins} ({rate_wins:.1%})")
print(f"Draws: {n_draws} ({rate_draws:.1%})")
print(f"Losses: {n_losses} ({rate_losses:.1%})")

if rate_losses > 0.2:
    print("Your are losing more than we expected!")
    print("You should be proud of yourself! :)")
